In [1]:
import numpy as np

In [2]:
tot_obs = np.load("vel_comm_obs.npy")
tot_actions = np.load("vel_comm_actions.npy")

In [3]:
tot_obs.shape, tot_actions.shape

((5000, 3, 18), (5000, 3, 1, 5))

## Extracting examples for agent and action 

In [4]:
AGENT = 0 
ACTION = 1
OPPOSITE_ACTION = 2 

In [5]:
#indexes for action
obs_by_act = [[],[],[],[],[]]
agent_acts = tot_actions[:,AGENT].squeeze()
agent_obs = tot_obs[:,AGENT]
for a, o in zip(agent_acts, agent_obs):
    obs_by_act[int((a[2]-a[1])>0 )+1].append(o)

In [28]:
delta_x = np.array(obs_by_act[ACTION])[:,4]
delta_y = np.array(obs_by_act[ACTION])[:,0]
opp_delta_x = np.array(obs_by_act[OPPOSITE_ACTION])[:,4]
opp_delta_y = np.array(obs_by_act[OPPOSITE_ACTION])[:,0]

In [29]:
def create_context(obs, string):
    s = "{"
    s+= f"{string[0]}({obs[0]}). "
    s+= f"{string[1]}({obs[1]})."
    s+= "}"
    return s 


In [30]:
# for x,y in zip(delta_x,delta_y):
    # print(create_context([x,y], ["delta_x", "delta_y"]))

## Discretizing obs

In [31]:
def discretize_int(data, multi=10):
    data = np.round(data*multi,0)
    return data.astype(int)

In [32]:
disc_delta_x = discretize_int(delta_x)
disc_delta_y = discretize_int(delta_y)
disc_opp_delta_x = discretize_int(opp_delta_x)
disc_opp_delta_y = discretize_int(opp_delta_y)

In [33]:
def get_action_string():
    act_dict = {
        0: "BHO",
        1: "left",
        2: "right",
        3: "down",
        4: "up"
    }
    return "{"+act_dict[ACTION]+"}"

In [34]:
get_action_string()

'{left}'

## Example parser

In [35]:
def write_example(obs, string, iter, pos=True):
    if pos:
        return f"#pos(ex{iter}, {get_action_string()}, {{}}, {create_context(obs, string)}).\n"
    else:
        return f"#pos(ex{iter}, {{}}, {get_action_string()}, {create_context(obs, string)}).\n"

In [36]:
# ex_set = set()
# for iter, obs in enumerate(zip(disc_delta_x,disc_delta_y)):
#     ex_set.add(write_example(obs,["delta_x", "delta_y"],1, True))
# for iter2, obs in enumerate(zip(disc_opp_delta_x,disc_opp_delta_y)):
#     ex_set.add(write_example(obs,["delta_x", "delta_y"],1, False))
# len(ex_set)

In [37]:
len(delta_x), len(disc_opp_delta_x)

(2577, 2423)

In [38]:
# with open(f"vel_comm_example_{ACTION}.txt", "w") as file: 
#     for ex in ex_set:
#         file.write(ex)


In [40]:
check_Set = set()
with open(f"vel_comm_example_{ACTION}.txt", "w") as file: 
    for iter, obs in enumerate(zip(disc_delta_x,disc_delta_y)):
        if not (*obs, ACTION) in check_Set:
            file.write(write_example(obs,["delta", "pos"],iter, True))
            check_Set.add((*obs, ACTION))
    for iter2, obs in enumerate(zip(disc_opp_delta_x,disc_opp_delta_y)):
        if not (*obs, ACTION) in check_Set:
            file.write(write_example(obs,["delta", "pos"],iter+iter2, False))
            check_Set.add((*obs, ACTION))
file.close()